In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import xlsxwriter
import time

%matplotlib qt

from sklearn.datasets import fetch_openml

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
# dataset

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target']

y = y.astype(np.uint8)
X_scale = X/255
x_train, x_test, y_train, y_test = X_scale[:60000], X_scale[60000:], y[:60000], y[60000:]
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

/Users/kyryll/miniforge3/envs/env1/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
EPOCHS = 10

static_f_time = []
adaptive_f_time = []
adaptive_nn_time = []
adaptive_conv_nn_time = []

In [5]:
# 39760
static_f_model = keras.Sequential([
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [6]:
# 39760
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(50, activation=tf.nn.swish),
    keras.layers.Dense(10, activation="softmax")
])

adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [7]:
# Adaptive Layer
# 39818
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(48, structure=[10, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.5),
    AdaptiveLayer(10, structure=[5, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.softmax, skip_w=0.5)
])

adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [8]:
# Adaptive Layer based on 1D Conv
# 39840
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(45, structure=[32, 8], split=1, inner_hidden_activation=tf.nn.tanh,
                  inner_out_activation=tf.nn.relu),
    AdaptiveLayerConv(10, structure=[64, 16], split=1, inner_hidden_activation=tf.nn.leaky_relu,
                  inner_out_activation=tf.nn.softmax)
])

adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [9]:
start = time.time()
static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
static_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 527us/step - loss: 0.3695 - accuracy: 0.8984
Epoch 2/10
938/938 [==============================] - 0s 525us/step - loss: 0.1858 - accuracy: 0.9470
Epoch 3/10
938/938 [==============================] - 0s 526us/step - loss: 0.1427 - accuracy: 0.9593
Epoch 4/10
938/938 [==============================] - 0s 526us/step - loss: 0.1169 - accuracy: 0.9664
Epoch 5/10
938/938 [==============================] - 0s 532us/step - loss: 0.0987 - accuracy: 0.9713
Epoch 6/10
938/938 [==============================] - 1s 577us/step - loss: 0.0868 - accuracy: 0.9744
Epoch 7/10
938/938 [==============================] - 0s 528us/step - loss: 0.0761 - accuracy: 0.9773
Epoch 8/10
938/938 [==============================] - 1s 534us/step - loss: 0.0674 - accuracy: 0.9796
Epoch 9/10
938/938 [==============================] - 0s 528us/step - loss: 0.0608 - accuracy: 0.9816
Epoch 10/10
938/938 [==============================] - 1s 533us/step - loss: 0.055

In [10]:
start = time.time()
adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 534us/step - loss: 0.3657 - accuracy: 0.8995
Epoch 2/10
938/938 [==============================] - 1s 537us/step - loss: 0.1935 - accuracy: 0.9435
Epoch 3/10
938/938 [==============================] - 1s 539us/step - loss: 0.1482 - accuracy: 0.9570
Epoch 4/10
938/938 [==============================] - 1s 533us/step - loss: 0.1196 - accuracy: 0.9650
Epoch 5/10
938/938 [==============================] - 1s 536us/step - loss: 0.0991 - accuracy: 0.9710
Epoch 6/10
938/938 [==============================] - 1s 537us/step - loss: 0.0851 - accuracy: 0.9748
Epoch 7/10
938/938 [==============================] - 1s 536us/step - loss: 0.0733 - accuracy: 0.9779
Epoch 8/10
938/938 [==============================] - 1s 545us/step - loss: 0.0653 - accuracy: 0.9809
Epoch 9/10
938/938 [==============================] - 1s 542us/step - loss: 0.0573 - accuracy: 0.9828
Epoch 10/10
938/938 [==============================] - 1s 542us/step - loss: 0.051

In [11]:
start = time.time()
adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 900us/step - loss: 0.4771 - accuracy: 0.8544
Epoch 2/10
938/938 [==============================] - 1s 895us/step - loss: 0.1621 - accuracy: 0.9521
Epoch 3/10
938/938 [==============================] - 1s 904us/step - loss: 0.1181 - accuracy: 0.9653
Epoch 4/10
938/938 [==============================] - 1s 903us/step - loss: 0.0942 - accuracy: 0.9722
Epoch 5/10
938/938 [==============================] - 1s 907us/step - loss: 0.0785 - accuracy: 0.9762
Epoch 6/10
938/938 [==============================] - 1s 908us/step - loss: 0.0691 - accuracy: 0.9788
Epoch 7/10
938/938 [==============================] - 1s 908us/step - loss: 0.0604 - accuracy: 0.9810
Epoch 8/10
938/938 [==============================] - 1s 906us/step - loss: 0.0531 - accuracy: 0.9833
Epoch 9/10
938/938 [==============================] - 1s 911us/step - loss: 0.0482 - accuracy: 0.9848
Epoch 10/10
938/938 [==============================] - 1s 915us/step - loss: 0.044

In [12]:
start = time.time()
adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_conv_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 1ms/step - loss: 0.4040 - accuracy: 0.8862
Epoch 2/10
938/938 [==============================] - 1s 1ms/step - loss: 0.1808 - accuracy: 0.9466
Epoch 3/10
938/938 [==============================] - 1s 1ms/step - loss: 0.1376 - accuracy: 0.9584
Epoch 4/10
938/938 [==============================] - 1s 1ms/step - loss: 0.1149 - accuracy: 0.9649
Epoch 5/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0972 - accuracy: 0.9700
Epoch 6/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0858 - accuracy: 0.9736
Epoch 7/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0732 - accuracy: 0.9771
Epoch 8/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0657 - accuracy: 0.9794
Epoch 9/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0601 - accuracy: 0.9811
Epoch 10/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0536 - accuracy: 0.9827

In [13]:
print("Static Function model : ", static_f_time[-1])
print("\nAdaptive Function model\ : ", adaptive_f_time[-1])
print("\nAdaptive NN model: ", adaptive_nn_time[-1])
print("\nAdaptive Conv NN model : ", adaptive_conv_nn_time[-1])

Static Function model :  5.383196830749512

Adaptive Function model\ :  5.372308015823364

Adaptive NN model:  9.068172216415405

Adaptive Conv NN model :  10.478008031845093


In [14]:
# 79510
static_f_model = keras.Sequential([
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [15]:
# 79510
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(100, activation=tf.nn.swish),
    keras.layers.Dense(10, activation="softmax")
])

adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [16]:
# Adaptive Layer
# 79710
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(97, structure=[8, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.5),
    AdaptiveLayer(10, structure=[5, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.softmax, skip_w=0.5)
])

adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [17]:
# Adaptive Layer based on 1D Conv
# 79132
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(92, structure=[32, 8], split=1, inner_hidden_activation=tf.nn.tanh,
                  inner_out_activation=tf.nn.relu),
    AdaptiveLayerConv(10, structure=[64, 16], split=1, inner_hidden_activation=tf.nn.leaky_relu,
                  inner_out_activation=tf.nn.softmax)
])

adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [18]:
start = time.time()
static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
static_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 600us/step - loss: 0.3128 - accuracy: 0.9136
Epoch 2/10
938/938 [==============================] - 1s 641us/step - loss: 0.1463 - accuracy: 0.9582
Epoch 3/10
938/938 [==============================] - 1s 642us/step - loss: 0.1036 - accuracy: 0.9699
Epoch 4/10
938/938 [==============================] - 1s 664us/step - loss: 0.0807 - accuracy: 0.9761
Epoch 5/10
938/938 [==============================] - 1s 616us/step - loss: 0.0627 - accuracy: 0.9815
Epoch 6/10
938/938 [==============================] - 1s 640us/step - loss: 0.0522 - accuracy: 0.9845
Epoch 7/10
938/938 [==============================] - 1s 630us/step - loss: 0.0426 - accuracy: 0.9872
Epoch 8/10
938/938 [==============================] - 1s 608us/step - loss: 0.0357 - accuracy: 0.9894
Epoch 9/10
938/938 [==============================] - 1s 638us/step - loss: 0.0301 - accuracy: 0.9907
Epoch 10/10
938/938 [==============================] - 1s 655us/step - loss: 0.024

In [19]:
start = time.time()
adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 630us/step - loss: 0.3339 - accuracy: 0.9074
Epoch 2/10
938/938 [==============================] - 1s 628us/step - loss: 0.1596 - accuracy: 0.9541
Epoch 3/10
938/938 [==============================] - 1s 624us/step - loss: 0.1096 - accuracy: 0.9689
Epoch 4/10
938/938 [==============================] - 1s 629us/step - loss: 0.0826 - accuracy: 0.9756
Epoch 5/10
938/938 [==============================] - 1s 627us/step - loss: 0.0641 - accuracy: 0.9813
Epoch 6/10
938/938 [==============================] - 1s 628us/step - loss: 0.0525 - accuracy: 0.9845
Epoch 7/10
938/938 [==============================] - 1s 624us/step - loss: 0.0432 - accuracy: 0.9873
Epoch 8/10
938/938 [==============================] - 1s 625us/step - loss: 0.0354 - accuracy: 0.9894
Epoch 9/10
938/938 [==============================] - 1s 621us/step - loss: 0.0291 - accuracy: 0.9912
Epoch 10/10
938/938 [==============================] - 1s 625us/step - loss: 0.024

In [20]:
start = time.time()
adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 1ms/step - loss: 0.4217 - accuracy: 0.8826
Epoch 2/10
938/938 [==============================] - 1s 1ms/step - loss: 0.1372 - accuracy: 0.9612
Epoch 3/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0954 - accuracy: 0.9718
Epoch 4/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0711 - accuracy: 0.9785
Epoch 5/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0559 - accuracy: 0.9828
Epoch 6/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0472 - accuracy: 0.9854
Epoch 7/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0399 - accuracy: 0.9873
Epoch 8/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0331 - accuracy: 0.9892
Epoch 9/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0311 - accuracy: 0.9897
Epoch 10/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0263 - accuracy: 0.9912

In [21]:
start = time.time()
adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_conv_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 1ms/step - loss: 0.3334 - accuracy: 0.9055
Epoch 2/10
938/938 [==============================] - 1s 1ms/step - loss: 0.1468 - accuracy: 0.9564
Epoch 3/10
938/938 [==============================] - 1s 1ms/step - loss: 0.1092 - accuracy: 0.9668
Epoch 4/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0873 - accuracy: 0.9731
Epoch 5/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0687 - accuracy: 0.9786
Epoch 6/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0610 - accuracy: 0.9806
Epoch 7/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0510 - accuracy: 0.9836
Epoch 8/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0436 - accuracy: 0.9852
Epoch 9/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0401 - accuracy: 0.9864
Epoch 10/10
938/938 [==============================] - 1s 1ms/step - loss: 0.0332 - accuracy: 0.9888

In [22]:
print("Static Function model : ", static_f_time[-1])
print("\nAdaptive Function model\ : ", adaptive_f_time[-1])
print("\nAdaptive NN model: ", adaptive_nn_time[-1])
print("\nAdaptive Conv NN model : ", adaptive_conv_nn_time[-1])

Static Function model :  6.3239758014678955

Adaptive Function model\ :  6.225214958190918

Adaptive NN model:  10.490297079086304

Adaptive Conv NN model :  11.543746948242188


In [23]:
# 125010
static_f_model = keras.Sequential([
    keras.layers.Dense(140, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [24]:
# 125010
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(140, activation=tf.nn.swish),
    keras.layers.Dense(100, activation=tf.nn.swish),
    keras.layers.Dense(10, activation="softmax")
])

adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [25]:
# Adaptive Layer
# 125000
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(135, structure=[6, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.5),
    AdaptiveLayer(100, structure=[5, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.5),
    AdaptiveLayer(10, structure=[8, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.softmax, skip_w=0.5)
])

adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [26]:
# Adaptive Layer based on 1D Conv
# 125054 
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(125, structure=[32, 16], split=1, inner_hidden_activation=tf.nn.tanh,
                  inner_out_activation=tf.nn.relu),
    AdaptiveLayerConv(100, structure=[32, 11], split=1, inner_hidden_activation=tf.nn.tanh,
                  inner_out_activation=tf.nn.relu),
    AdaptiveLayerConv(10, structure=[64, 15], split=1, inner_hidden_activation=tf.nn.leaky_relu,
                  inner_out_activation=tf.nn.softmax)
])

adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [27]:
start = time.time()
static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
static_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 791us/step - loss: 0.2635 - accuracy: 0.9236
Epoch 2/10
938/938 [==============================] - 1s 814us/step - loss: 0.1076 - accuracy: 0.9675
Epoch 3/10
938/938 [==============================] - 1s 835us/step - loss: 0.0748 - accuracy: 0.9772
Epoch 4/10
938/938 [==============================] - 1s 806us/step - loss: 0.0569 - accuracy: 0.9825
Epoch 5/10
938/938 [==============================] - 1s 792us/step - loss: 0.0431 - accuracy: 0.9863
Epoch 6/10
938/938 [==============================] - 1s 788us/step - loss: 0.0371 - accuracy: 0.9882
Epoch 7/10
938/938 [==============================] - 1s 783us/step - loss: 0.0273 - accuracy: 0.9914
Epoch 8/10
938/938 [==============================] - 1s 784us/step - loss: 0.0242 - accuracy: 0.9921
Epoch 9/10
938/938 [==============================] - 1s 789us/step - loss: 0.0203 - accuracy: 0.9931
Epoch 10/10
938/938 [==============================] - 1s 795us/step - loss: 0.014

In [28]:
start = time.time()
adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 835us/step - loss: 0.2758 - accuracy: 0.9223
Epoch 2/10
938/938 [==============================] - 1s 831us/step - loss: 0.1099 - accuracy: 0.9671
Epoch 3/10
938/938 [==============================] - 1s 838us/step - loss: 0.0739 - accuracy: 0.9770
Epoch 4/10
938/938 [==============================] - 1s 843us/step - loss: 0.0548 - accuracy: 0.9826
Epoch 5/10
938/938 [==============================] - 1s 840us/step - loss: 0.0405 - accuracy: 0.9871
Epoch 6/10
938/938 [==============================] - 1s 844us/step - loss: 0.0320 - accuracy: 0.9897
Epoch 7/10
938/938 [==============================] - 1s 849us/step - loss: 0.0250 - accuracy: 0.9915
Epoch 8/10
938/938 [==============================] - 1s 849us/step - loss: 0.0199 - accuracy: 0.9936
Epoch 9/10
938/938 [==============================] - 1s 849us/step - loss: 0.0171 - accuracy: 0.9943
Epoch 10/10
938/938 [==============================] - 1s 844us/step - loss: 0.014

In [29]:
start = time.time()
adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 0.3731 - accuracy: 0.8883
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.1138 - accuracy: 0.9660
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0794 - accuracy: 0.9758
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0601 - accuracy: 0.9813
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0489 - accuracy: 0.9842
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0406 - accuracy: 0.9871
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0343 - accuracy: 0.9887
Epoch 8/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0310 - accuracy: 0.9895
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0279 - accuracy: 0.9908
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0233 - accuracy: 0.9923

In [30]:
start = time.time()
adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_conv_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 0.3110 - accuracy: 0.9090
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.1301 - accuracy: 0.9607
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0926 - accuracy: 0.9713
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0721 - accuracy: 0.9773
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0594 - accuracy: 0.9811
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0498 - accuracy: 0.9844
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0426 - accuracy: 0.9863
Epoch 8/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0390 - accuracy: 0.9873
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0314 - accuracy: 0.9893
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0281 - accuracy: 0.9908

In [31]:
print("Static Function model : ", static_f_time[-1])
print("\nAdaptive Function model\ : ", adaptive_f_time[-1])
print("\nAdaptive NN model: ", adaptive_nn_time[-1])
print("\nAdaptive Conv NN model : ", adaptive_conv_nn_time[-1])

Static Function model :  7.887835264205933

Adaptive Function model\ :  8.306852102279663

Adaptive NN model:  17.647740840911865

Adaptive Conv NN model :  19.107141971588135


In [32]:
# 160410
static_f_model = keras.Sequential([
    keras.layers.Dense(180, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [33]:
# 160410
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(180, activation=tf.nn.swish),
    keras.layers.Dense(100, activation=tf.nn.swish),
    keras.layers.Dense(10, activation="softmax")
])

adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [34]:
# Adaptive Layer
# 160020
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(175, structure=[4, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.5),
    AdaptiveLayer(100, structure=[5, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.leaky_relu, skip_w=0.5),
    AdaptiveLayer(10, structure=[5, 1], inner_hidden_activation=tf.nn.relu, 
                  inner_out_activation=tf.nn.softmax, skip_w=0.5)
])

adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [35]:
# Adaptive Layer based on 1D Conv
# 160442
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(165, structure=[32, 12], split=1, inner_hidden_activation=tf.nn.tanh,
                  inner_out_activation=tf.nn.relu),
    AdaptiveLayerConv(100, structure=[32, 10], split=1, inner_hidden_activation=tf.nn.tanh,
                  inner_out_activation=tf.nn.relu),
    AdaptiveLayerConv(10, structure=[32, 10], split=1, inner_hidden_activation=tf.nn.leaky_relu,
                  inner_out_activation=tf.nn.softmax)
])

adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

In [36]:
start = time.time()
static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
static_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 884us/step - loss: 0.2456 - accuracy: 0.9286
Epoch 2/10
938/938 [==============================] - 1s 883us/step - loss: 0.0980 - accuracy: 0.9702
Epoch 3/10
938/938 [==============================] - 1s 887us/step - loss: 0.0669 - accuracy: 0.9789
Epoch 4/10
938/938 [==============================] - 1s 878us/step - loss: 0.0490 - accuracy: 0.9849
Epoch 5/10
938/938 [==============================] - 1s 874us/step - loss: 0.0366 - accuracy: 0.9886
Epoch 6/10
938/938 [==============================] - 1s 874us/step - loss: 0.0298 - accuracy: 0.9906
Epoch 7/10
938/938 [==============================] - 1s 882us/step - loss: 0.0243 - accuracy: 0.9921
Epoch 8/10
938/938 [==============================] - 1s 875us/step - loss: 0.0213 - accuracy: 0.9927
Epoch 9/10
938/938 [==============================] - 1s 878us/step - loss: 0.0145 - accuracy: 0.9954
Epoch 10/10
938/938 [==============================] - 1s 876us/step - loss: 0.016

In [37]:
start = time.time()
adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_f_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 1s 933us/step - loss: 0.2643 - accuracy: 0.9236
Epoch 2/10
938/938 [==============================] - 1s 955us/step - loss: 0.1042 - accuracy: 0.9692
Epoch 3/10
938/938 [==============================] - 1s 946us/step - loss: 0.0704 - accuracy: 0.9786
Epoch 4/10
938/938 [==============================] - 1s 946us/step - loss: 0.0506 - accuracy: 0.9835
Epoch 5/10
938/938 [==============================] - 1s 947us/step - loss: 0.0372 - accuracy: 0.9882
Epoch 6/10
938/938 [==============================] - 1s 951us/step - loss: 0.0298 - accuracy: 0.9903
Epoch 7/10
938/938 [==============================] - 1s 956us/step - loss: 0.0227 - accuracy: 0.9925
Epoch 8/10
938/938 [==============================] - 1s 947us/step - loss: 0.0199 - accuracy: 0.9935
Epoch 9/10
938/938 [==============================] - 1s 942us/step - loss: 0.0129 - accuracy: 0.9957
Epoch 10/10
938/938 [==============================] - 1s 955us/step - loss: 0.013

In [38]:
start = time.time()
adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 0.3621 - accuracy: 0.8885
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.1137 - accuracy: 0.9664
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0797 - accuracy: 0.9747
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0582 - accuracy: 0.9817
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0484 - accuracy: 0.9844
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0402 - accuracy: 0.9870
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0356 - accuracy: 0.9884
Epoch 8/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0313 - accuracy: 0.9899
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0240 - accuracy: 0.9919
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0234 - accuracy: 0.9923

In [39]:
start = time.time()
adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)
end = time.time()
adaptive_conv_nn_time.append(end-start)

Epoch 1/10
938/938 [==============================] - 2s 2ms/step - loss: 0.3040 - accuracy: 0.9088
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.1250 - accuracy: 0.9626
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0876 - accuracy: 0.9727
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0677 - accuracy: 0.9790
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0551 - accuracy: 0.9827
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0467 - accuracy: 0.9857
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0392 - accuracy: 0.9869
Epoch 8/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0362 - accuracy: 0.9886
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0288 - accuracy: 0.9906
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.0254 - accuracy: 0.9916

In [40]:
print("Static Function model : ", static_f_time[-1])
print("\nAdaptive Function model\ : ", adaptive_f_time[-1])
print("\nAdaptive NN model: ", adaptive_nn_time[-1])
print("\nAdaptive Conv NN model : ", adaptive_conv_nn_time[-1])

Static Function model :  8.528810024261475

Adaptive Function model\ :  9.188874006271362

Adaptive NN model:  18.503164291381836

Adaptive Conv NN model :  20.2519690990448


In [41]:
sizes = [40000, 80000, 120000, 160000]

In [42]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot()
plt.title("Parameters - Time", fontsize="xx-large")

ax.set_ylabel("time (sec)", fontsize="large")
ax.set_xlabel('Number of parameters', fontsize="large")

ax.plot(sizes, static_f_time, label='ReLU Function')
ax.plot(sizes, adaptive_f_time, label='Swish Function')
ax.plot(sizes, adaptive_nn_time, label='Adaptive NN')
ax.plot(sizes, adaptive_conv_nn_time, label='Adaptive Conv NN')

plt.xticks(sizes)
    
ax.legend()
    
plt.grid()
plt.show()

In [43]:
workbook = xlsxwriter.Workbook('time_measurements.xlsx')
parameters_num = [40000, 80000, 120000, 160000] 
worksheet = workbook.add_worksheet('time_measurements')
    
worksheet.write(1, 1, "Number of parameters")
worksheet.write(1, 2, "ReLU Function")
worksheet.write(1, 3, "Swish Function")
worksheet.write(1, 4, "Adaptive NN")
worksheet.write(1, 5, "Adaptive Conv NN")
    
for i in range(0, len(parameters_num)):
    worksheet.write(2+i, 1, parameters_num[i])
    worksheet.write(2+i, 2, static_f_time[i])
    worksheet.write(2+i, 3, adaptive_f_time[i])
    worksheet.write(2+i, 4, adaptive_nn_time[i])
    worksheet.write(2+i, 5, adaptive_conv_nn_time[i])
 
workbook.close()